# TDX 下載函數

In [41]:
import os
import requests
import json

from datetime import datetime # 輸出下載日期用
import time # 下載間隔，避免短時間下載被拒絕


def tdx_data_download(app_id, app_key, url, output_folder, output_file_name):
    """
    從 TDX 下載資料並輸出到指定資料夾與檔名。
    """

    # 取得 Token
    auth_url = "https://tdx.transportdata.tw/auth/realms/TDXConnect/protocol/openid-connect/token"

    auth_header = {
        'content-type': 'application/x-www-form-urlencoded'
    }

    auth_data = {
        'grant_type': 'client_credentials',
        'client_id': app_id,
        'client_secret': app_key
    }

    auth_response = requests.post(auth_url, headers=auth_header, data=auth_data)
    token = auth_response.json().get("access_token")

    # 下載資料
    data_header = {
        'authorization': f'Bearer {token}',
        'Accept-Encoding': 'gzip'
    }

    data_response = requests.get(url, headers=data_header)

    # 確保資料夾存在
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 寫入檔案
    output_path = os.path.join(output_folder, output_file_name)
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(data_response.text)

    print(f"資料已下載並存儲到 {output_path} 中。")
    return output_path

def create_folder(folder_name):
    """建立資料夾"""
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    return os.path.abspath(folder_name)

def get_app_credentials(file_path):
    """從設定檔案讀取 TDX 的 app_id 和 app_key"""
    import pandas as pd
    # 設定檔案路徑
    file_path = r"D:\常用Tool\tdx帳密.txt"  #需存於程式碼相同資料夾中
    # 讀取文件內容
    with open(file_path, 'r', encoding='utf-8') as file:
        file_content = file.read()
    path_dict = {}
    for line in file_content.strip().split('\n'):
        key, value = line.split('=')
        path_dict[key.strip()] = value.strip()
        # path_dict[key.strip()] = value.strip().strip("'")
    # 將字典轉換為DataFrame
    info_table = pd.DataFrame(list(path_dict.items()), columns=['Item', 'Value'])

    app_id = info_table[info_table['Item'] == 'app_id']['Value'].iloc[0]
    app_key = info_table[info_table['Item'] == 'app_key']['Value'].iloc[0]

    return app_id, app_key


In [ ]:
app_id, app_key = get_app_credentials(r"D:\常用Tool\tdx帳密.txt") # 讀取TDX帳密 也可以自己手key in
busstopseq_folder = create_folder(os.path.join(os.getcwd(), '..', "00_TDX資料下載", "01公車站序資料"))
busroute_folder = create_folder(os.path.join(os.getcwd(), '..', "00_TDX資料下載", "02公車路線資料"))

selectcity = {'新北市':'NewTaipei', '臺北市':'Taipei', '基隆市':'Keelung', '桃園市':'Taoyuan'}
downloaddate = datetime.today().strftime("%Y-%m-%d")

'''下載公車站序資料'''
for city, city_en in selectcity.items():
    url = f"https://tdx.transportdata.tw/api/basic/v2/Bus/StopOfRoute/City/{city_en}?%24format=XML"
    output_folder = busstopseq_folder
    output_file_name = f"公車站序資料_{city}_{downloaddate}.xml"
    tdx_data_download(
        app_id = app_id,
        app_key = app_key,
        url = url,
        output_folder = output_folder,
        output_file_name = output_file_name
        )
    
    time.sleep(60)

'''下載公車路線shp資料'''
for city, city_en in selectcity.items():
    # url = f"https://tdx.transportdata.tw/api/basic/v2/Bus/StopOfRoute/City/{city_en}?%24format=XML"
    url = f"https://tdx.transportdata.tw/api/basic/v2/Bus/Shape/City/{city_en}?%24format=XML"
    output_folder = busroute_folder
    output_file_name = f"市區公車路線資料_{city}_{downloaddate}.xml"
    tdx_data_download(
        app_id = app_id,
        app_key = app_key,
        url = url,
        output_folder = output_folder,
        output_file_name = output_file_name
        )
    
    time.sleep(60)